In [354]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [355]:
data = pd.read_csv('..\Data\data_mod.csv')

In [356]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           864 non-null    int64  
 1   Trimestre                     864 non-null    int64  
 2   Fecha                         864 non-null    object 
 3   Provincia                     864 non-null    object 
 4   Banda ancha fija              864 non-null    int64  
 5   Dial up                       864 non-null    float64
 6   Mbps (Media de bajada)        864 non-null    float64
 7   ADSL                          864 non-null    int64  
 8   Cablemodem                    864 non-null    int64  
 9   Fibra óptica                  864 non-null    int64  
 10  Wireless                      864 non-null    int64  
 11  Otros                         864 non-null    int64  
 12  Total                         864 non-null    int64  
 13  Acces

## -------------------------------------------------------------------------------

In [357]:
data

,Año,Trimestre,Fecha,Provincia,Banda ancha fija,Dial up,Mbps (Media de bajada),ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Accesos por cada 100 hogares
0,2022,4,2022-10-01,Buenos Aires,4788466,6199.0,111.000000,325908,2767247,1499149,129910,72451,4794665,78.940000
1,2022,4,2022-10-01,Capital Federal,1549781,2145.0,182.860000,138753,1238196,138548,4603,31826,1551926,122.730000
2,2022,4,2022-10-01,Catamarca,70908,1.0,83.330000,8952,10998,47337,1379,2243,70909,65.770000
3,2022,4,2022-10-01,Chaco,143658,5.0,104.020000,24582,61265,47285,7957,2574,143663,43.620000
4,2022,4,2022-10-01,Chubut,167855,904.0,15.920000,45450,73923,9815,30721,8850,168759,85.570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,2014,1,2014-01-01,Santa Cruz,26304,460.0,3.315991,18927,100,381,4092,3264,26764,28.939093
860,2014,1,2014-01-01,Santa Fe,506000,612.0,3.147407,322662,174296,3059,5951,644,506612,47.847663
861,2014,1,2014-01-01,Santiago Del Estero,37124,9.0,2.949472,32567,3598,19,915,34,37133,16.309924
862,2014,1,2014-01-01,Tierra Del Fuego,27272,766.0,3.446888,21618,2837,648,1,2934,28038,63.965926


In [358]:
grouped1 = data.groupby(['Año','Trimestre', 'Provincia'])[['Banda ancha fija', 'Dial up','Mbps (Media de bajada)']].sum().reset_index()



In [359]:
grouped = data.groupby(['Año','Trimestre'])[['Banda ancha fija', 'Dial up']].sum().reset_index()


In [360]:
grouped5 = data.groupby(['Año'])[['Cablemodem','Fibra óptica','Wireless','ADSL','Otros']].sum().reset_index()
grouped5

,Año,Cablemodem,Fibra óptica,Wireless,ADSL,Otros
0,2014,9975087,604862,305234,14884750,286700
1,2015,11162971,620877,331431,15115694,242846
2,2016,12177088,680127,337396,15073814,227633
3,2017,14148063,766312,432844,14639302,344162
4,2018,16571272,1079124,870047,13981984,465760
5,2019,18754974,2450177,1262831,12188858,909341
6,2020,20539964,4636620,1563813,9072828,958018
7,2021,22877010,6473506,1927038,7802044,1006793
8,2022,24122067,10874934,2232136,5712223,940575


In [361]:
datap = pd.read_csv('..\Data\poblacion_identificada.csv')
grup = datap.groupby(['provincia_nombre'])['cantidad'].sum().reset_index()
grup.rename(columns={'provincia_nombre':'Provincia', 'cantidad':'Población'}, inplace=True)
grup.replace('CIUDAD DE BUENOS AIRES', 'Capital Federal', inplace=True)
grup.replace('CORDOBA', 'Córdoba', inplace=True)
grup.replace('ENTRE RIOS', 'Entre Ríos', inplace=True)
grup.replace('NEUQUEN', 'Neuquén', inplace=True)
grup.replace('RIO NEGRO', 'Río Negro', inplace=True)
grup.replace('TUCUMAN', 'Tucumán', inplace=True)
grup['Provincia'] = grup['Provincia'].str.title()
grup.sort_values('Provincia',inplace=True)
#grup.to_excel('poblacion_argentina.xlsx', index=False)

In [362]:
filtro_data = data[(data['Año'] == 2022) & (data['Trimestre'] == 4)]
grupo_data = filtro_data.groupby('Provincia').agg({'Total': 'sum', 'Mbps (Media de bajada)': 'mean'}).reset_index()
# grupo_data.to_excel('acceso_provincia_2022.xlsx', index=False)
merged_data = grupo_data.merge(grup, on='Provincia')
merged_data['% Cobertura'] = round(((merged_data['Total']*100)/merged_data['Población']),2)

In [363]:
def calculate_kpi1(group1):
    group1['KPI_1'] = round((group1['Accesos por cada 100 hogares'].diff()/group1['Accesos por cada 100 hogares'])*100,2)
    return group1

data.sort_values('Fecha', inplace=True)
data = data.groupby('Provincia').apply(calculate_kpi1)

data.reset_index(drop=True, inplace=True)

def calculate_kpi2(group2):
    group2['KPI_2'] = round((group2['Mbps (Media de bajada)'].diff()/group2['Mbps (Media de bajada)'])*100,2)
    return group2

data = data.groupby('Provincia').apply(calculate_kpi2)

data.reset_index(drop=True, inplace=True)

data.to_csv('..\Data\data_internet.csv', index=False)

In [364]:
data

,Año,Trimestre,Fecha,Provincia,Banda ancha fija,Dial up,Mbps (Media de bajada),ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Accesos por cada 100 hogares,KPI_1,KPI_2
0,2014,1,2014-01-01,Buenos Aires,2725732,14144.0,3.733133,1567685,1000879,120960,16528,33824,2739876,54.570073,NaN,NaN
1,2014,2,2014-04-01,Buenos Aires,2750430,14116.0,3.851863,1568881,1025490,120008,17020,33147,2764546,55.061417,0.89,3.08
2,2014,3,2014-07-01,Buenos Aires,2835537,14028.0,3.969047,1562533,1110628,124474,20049,31881,2849565,56.754739,2.98,2.95
3,2014,4,2014-10-01,Buenos Aires,2801178,15229.0,4.198944,1575978,1070135,120528,17728,32038,2816407,56.094337,-1.18,5.48
4,2015,1,2015-01-01,Buenos Aires,2847076,14766.0,4.323804,1583135,1096836,134177,18754,28940,2861842,56.370484,0.49,2.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,2021,4,2021-10-01,Tucumán,238609,30.0,45.148122,97830,59022,78790,2511,486,238639,55.925991,2.31,6.75
860,2022,1,2022-01-01,Tucumán,245554,30.0,47.240000,92532,62396,86174,3995,487,245584,57.390032,2.55,4.43
861,2022,2,2022-04-01,Tucumán,254497,30.0,50.090000,85893,15327,148823,3997,487,254527,59.310000,3.24,5.69
862,2022,3,2022-07-01,Tucumán,263087,30.0,53.360000,77738,18518,162345,3997,519,263117,61.140000,2.99,6.13


In [365]:
filtro_data1 = data[(data['Año'] == 2022) & (data['Trimestre'] == 4)]
data1 = filtro_data1[['Provincia','Accesos por cada 100 hogares','KPI_1','KPI_2']]


In [366]:
merged_data2 = merged_data.merge(data1, on='Provincia')
merged_data2.to_csv('..\Data\data_4_2022.csv', index=False)